In [1]:
%pip install scikit-learn pandas fastapi uvicorn nest_asyncio


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\rrdas\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import pickle

# Sample data
df = pd.DataFrame({
    'amount': [100, 2000, 150, 3000, 50],
    'location': ['US', 'NG', 'US', 'RU', 'US'],
    'timestamp': pd.date_range(start='2023-01-01', periods=5, freq='H'),
    'fraud': [0, 1, 0, 1, 0]
})

# Feature engineering
df['is_foreign'] = df['location'] != 'US'
df['hour'] = df['timestamp'].dt.hour
X = df[['amount', 'is_foreign', 'hour']]
X['is_foreign'] = X['is_foreign'].astype(int)
y = df['fraud']

# Train model
clf = RandomForestClassifier()
clf.fit(X, y)

# Save model
with open("fraud_model.pkl", "wb") as f:
    pickle.dump(clf, f)


C:\Users\rrdas\AppData\Local\Temp\ipykernel_2496\682612250.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  'timestamp': pd.date_range(start='2023-01-01', periods=5, freq='H'),
C:\Users\rrdas\AppData\Local\Temp\ipykernel_2496\682612250.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['is_foreign'] = X['is_foreign'].astype(int)


In [3]:
def preprocess(data):
    df = pd.DataFrame([data])
    df['is_foreign'] = df['location'] != 'US'
    df['hour'] = pd.to_datetime(df['timestamp']).dt.hour
    df['is_foreign'] = df['is_foreign'].astype(int)
    return df[['amount', 'is_foreign', 'hour']]

def load_model():
    with open("fraud_model.pkl", "rb") as f:
        return pickle.load(f)

model = load_model()

def predict(data):
    features = preprocess(data)
    return bool(model.predict(features)[0])


In [4]:
from fastapi import FastAPI
from pydantic import BaseModel
import nest_asyncio
import uvicorn

nest_asyncio.apply()

app = FastAPI()

class Transaction(BaseModel):
    amount: float
    location: str
    timestamp: str

@app.post("/predict")
def predict_fraud(tx: Transaction):
    result = predict(tx.dict())
    return {"fraud": result}

# Run API inside notebook
uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [2496]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [2496]


In [5]:
import requests

tx = {
    "amount": 2500,
    "location": "RU",
    "timestamp": "2023-01-01T13:00:00"
}

response = requests.post("http://localhost:8000/predict", json=tx)
print(response.json())


Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at C:\Users\rrdas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\uvicorn\server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "C:\Users\rrdas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\uvicorn\main.py", line 579, in run
    server.run()
  File "C:\Users\rrdas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\uvicorn\server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\rrdas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\nest_asyncio.py", line 30, in 

ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000216B97B0050>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))